In [7]:
# %reset -f
from contextlib import closing
import sklearn, tflearn
from sklearn.utils import shuffle
from sklearn import metrics, preprocessing
from datetime import datetime
from dateutil.relativedelta import relativedelta

import tensorflow as tf, os, csv, sqlite3, pandas as pd, numpy as np, codecs, re, traceback, math, pickle
import matplotlib.pyplot as plt
np.set_printoptions(precision=7, suppress=True, linewidth=200)

## load column meta

In [8]:
ctxPath = '.'

def connect():
    return sqlite3.connect('mp.sqlite')

def loadVar(fpath):
    with open(fpath, 'rb') as r:
        return pickle.load(r)   

def saveMeta():
    df = pd.read_excel('{}/小額代收代付變數清單.xlsx'.format(ctxPath), sheetname='變數清單')
    df.drop(df.columns[range(6, len(df.columns))], axis=1, inplace=True)
    df.columns = ('idx', 'var_type', 'col_name', 'dscr', 'logic', 'if_null')
    df = df[df[u'var_type'] != u'衍生'].reset_index(drop=True)
    df.loc[len(df)] = df.iloc[1]
    df.drop(1, inplace=True)
    with closing(connect()) as conn:
        df.to_sql('mp_meta', conn, index=False, if_exists='replace')
    return df

def loadMeta():
    with closing(connect()) as conn:
        meta = pd.read_sql('select * from mp_meta', conn)
    return meta

# saveMeta()
meta = loadMeta()
meta.head(10)

,idx,var_type,col_name,dscr,logic,if_null
0,0,None,SUBSCR_ID,用戶編號,None,None
1,2,原始,LST1_CNT_BM,最近一個月月租銷貨次數,None,NULL補0
2,3,原始,LST1_CNT_BT,最近一個月計次銷貨次數,None,NULL補0
3,4,原始,LST1_CNT_FLUR,最近一個月失敗銷貨次數,None,NULL補0
4,5,原始,LST1_CNT_FLUR_UNDR_ACCT,最近一個月額度不足失敗銷貨次數,None,NULL補0
5,6,原始,LST1_CNT_FLUR_UPON_ACCT,最近一個月非額度不足失敗銷貨次數,None,NULL補0
6,7,原始,LST1_CNT_SCCESS,最近一個月成功銷貨次數,None,NULL補0
7,8,原始,LST1_GG_CNT,最近一個月GOOGLE銷貨次數,None,NULL補0
8,9,原始,LST1_GG_CNT_FLUR,最近一個月GOOGLE失敗銷貨次數,None,NULL補0
9,10,原始,LST1_GG_CNT_FLUR_UNDR_ACCT,最近一個月GOOGLE額度不足失敗銷貨次數,None,NULL補0


## save to sqlite, generate normalizer, 已過濾不想要的衍生變數

In [3]:
def genCsvNames(sdate, months=13):
    d = None
    for _ in range(months):
        d = sdate if d is None else d + relativedelta(months=1)
        yield 'MP_{}.csv'.format(d.strftime("%Y%m"))

def saveToSqlite(meta, csvFiles):
    colsidx = range(len(meta))
    adjidx = [colsidx[0]] + colsidx[2:] + [colsidx[1]]
    colNames = meta.col_name.values
    with closing(connect()) as conn:
        for e in csvFiles:
            p = '/'.join([ctxPath, e])
            df = pd.read_csv(p, header=None, usecols=meta.idx.values)
            df = df.iloc[:, adjidx]
            df.columns = colNames
            df.to_sql(e.replace('.csv', ''), conn, index=False, if_exists='replace')
            conn.executescript("create index {}_idx on {}(subscr_id);".format(e))
            print("{} done".format(e))
            
def genScaler(meta, name):
    with closing(connect()) as conn:
        df = pd.read_sql("select * from {}".format(name), conn)
    with open('/'.join([ctxPath, 'scaler_{}.pkl'.format(name)]), 'wb') as w:
        scaler = preprocessing.StandardScaler()
        scaler.fit(df.iloc[:, 1:-1].values)
        pickle.dump(scaler, w)

In [ ]:
saveToSqlite(meta, tuple(genCsvNames(datetime(2015, 7, 1), months=13)))
# genScaler(meta, 'mp_201508')

## 收集 train test 4個月 subid 交集

In [ ]:
def itsctSubIds(tablist, targetTab):
    subIds = set()
    with closing(connect()) as conn:
        for x, tab in enumerate(tablist):
            subid = set(pd.read_sql("select subscr_id from {}".format(tab), conn)['SUBSCR_ID'])
            if x == 0:
                subIds.update(subid)
            else:
                subIds &= subid
        pd.DataFrame({'subscr_id': list(subIds)[:100000]}).to_sql(targetTab, conn, if_exists='replace', index=False)
        conn.executescript("create index {0}_idx on {0}(subscr_id);".format(targetTab))
        return subIds
        
a = itsctSubIds(('mp_201507', 'mp_201508', 'mp_201509', 'mp_201510'), "train_subs")
b = itsctSubIds(('mp_201603', 'mp_201604', 'mp_201605', 'mp_201606'), "test_subs")

## MP NES(By Week) data save to sqlite

In [30]:
import csv
trainPath = "{}/mp/MP_NES_201510.csv".format(ctxPath)
trDf = pd.read_csv(trainPath, encoding="utf-8", header=None)
data = trDf.values
data = np.c_[data[:, 0], data[:, 2:]]
trDf = pd.DataFrame(columns=["SUBSCR_ID"] + list(range(72)), data=data)
trDf

,SUBSCR_ID,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,3628941,32,32,32,32,21,21,20,20,32,...,0,0,0,0,0,0,0,0,0,0
1,59358138,20,20,20,20,20,20,20,20,21,...,0,360,0,0,0,360,0,0,0,0
2,55232644,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
3,55200432,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
4,56406927,32,32,32,32,32,32,31,21,21,...,0,0,0,0,0,0,0,0,0,0
5,7618251,21,20,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
6,9145211,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
7,59496964,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
8,53241830,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
9,15652404,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0


In [31]:
testPath = "NES_by_week/MP_NES_201606.csv".format(ctxPath)
teDf = pd.read_csv(trainPath, encoding="utf-8", header=None)
data = teDf.values
data = np.c_[data[:, 0], data[:, 2:]]
teDf = pd.DataFrame(columns=["SUBSCR_ID"] + list(range(72)), data=data)
teDf

,SUBSCR_ID,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,3628941,30,21,21,20,21,20,30,21,20,...,0,0,0,0,0,240,0,0,0,0
1,59358138,32,32,32,32,32,32,32,32,20,...,0,2540,812,0,0,0,0,5981,10694,269
2,55232644,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
3,59948669,32,32,20,20,32,32,32,32,32,...,0,30,0,30,0,0,0,0,0,0
4,56406927,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
5,59176387,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
6,54109642,32,32,32,32,32,32,32,32,32,...,0,0,0,0,360,0,0,0,0,0
7,56068487,32,20,20,20,32,30,21,21,21,...,0,0,40,0,0,1000,0,0,295,1015
8,53241830,32,32,32,32,32,32,32,32,32,...,0,0,0,0,0,0,0,0,0,0
9,53475815,32,32,21,21,20,20,20,32,21,...,0,0,300,0,200,300,0,900,0,0


In [34]:
with closing(connect()) as conn:
    trDf.to_sql("mp_nes_201510", conn, index=False, if_exists="replace")
    teDf.to_sql("mp_nes_201606", conn, index=False, if_exists="replace")
    conn.commit()

## MP NES(By Month) data save to sqlite

In [28]:
ctxPath = '.'
with closing(connect()) as conn:
    for fname in ("MP_NES_201507M.csv", "MP_NES_201508M.csv", "MP_NES_201509M.csv", "MP_NES_201510M.csv",
            "MP_NES_201603M.csv", "MP_NES_201604M.csv", "MP_NES_201605M.csv", "MP_NES_201606M.csv"):
        df = pd.read_csv("{}/NES_by_month/{}".format(ctxPath, fname), header=None)
        df = df.drop(1, axis=1).where(df.notnull(), None) # .rename(columns=("SUBSCR_ID", "NES", "N_PAY", "AMT"))
        df.columns = ("SUBSCR_ID", "NES", "N_PAY", "AMT")
        tabname = "mp_nes_by_month_{}".format(fname.replace("MP_NES_", "").replace("M.csv", ""))
        df.to_sql(tabname, conn, index=False, if_exists="replace")
    conn.commit()